In [1]:
!pip install -q transformers
!pip install -q seqeval

     |████████████████████████████████| 1.9MB 22.4MB/s 
     |████████████████████████████████| 3.2MB 49.7MB/s 
     |████████████████████████████████| 890kB 47.0MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 


In [2]:
!git clone https://github.com/Ryzhtus/master-thesis

Cloning into 'master-thesis'...
remote: Enumerating objects: 2170, done.
remote: Counting objects: 100% (2170/2170), done.
remote: Compressing objects: 100% (2111/2111), done.
remote: Total 2170 (delta 91), reused 2135 (delta 56), pack-reused 0
Receiving objects: 100% (2170/2170), 2.59 MiB | 19.65 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [3]:
cd master-thesis

/content/master-thesis


In [4]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
!unrar x /gdrive/MyDrive/Диссертация/ontonotes.rar /content/master-thesis

Streaming output truncated to the last 5000 lines.
Extracting  /content/master-thesis/ontonotes/train/train_document55.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document550.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5500.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5501.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5502.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5503.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5504.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5505.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5506.txt      85%  OK 
Extracting  /content/master-thesis/ontonotes/train/train_document5507.txt     

In [6]:
from transformers import BertTokenizer
import torch

import warnings
warnings.filterwarnings('ignore')

In [7]:
from named_entity_recognition.dataset import DatasetNER, create_dataset_and_dataloader
from named_entity_recognition.model import BertNER
from named_entity_recognition.train import train_model, test_model

In [8]:
TOKENIZER = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
DEVICE = 'cuda' if torch.cuda.is_available else 'cpu'
EPOCHS = 4
BATCH_SIZE = 16

In [9]:
train_dataset, train_dataloader = create_dataset_and_dataloader('ontonotes', "/content/master-thesis/ontonotes/train", BATCH_SIZE, TOKENIZER)
eval_dataset, eval_dataloader = create_dataset_and_dataloader('ontonotes', "/content/master-thesis/ontonotes/development", BATCH_SIZE, TOKENIZER)
test_dataset, test_dataloader = create_dataset_and_dataloader('ontonotes', "/content/master-thesis/ontonotes/test", BATCH_SIZE, TOKENIZER)

In [10]:
eval_dataset.idx2tag = train_dataset.idx2tag
eval_dataset.tag2idx = train_dataset.tag2idx
test_dataset.idx2tag = train_dataset.idx2tag
test_dataset.tag2idx = train_dataset.tag2idx

In [ ]:
from named_entity_recognition.metrics import FMeasureStorage, AccuracyStorage
from seqeval.metrics import performance_measure
from tqdm import tqdm

import torch
import torch.nn as nn

def clear_tags(labels, predictions, masks, idx2tag, batch_element_length):
    """ this function removes <PAD>, CLS and SEP tags at each sentence
        and convert both ids of tags and batch elements to SeqEval input format
        [[first sentence tags], [second sentence tags], ..., [last sentence tags]]"""

    clear_labels = []
    clear_predictions = []
    masked_true_labels = []
    masked_pred_labels = []

    sentence_labels = []
    sentence_predictions = []
    sentence_true_labels_mask = []
    sentence_pred_labels_mask = []

    sentence_length = 0

    for idx in range(len(labels)):
        if labels[idx] != 0:
            sentence_labels.append(idx2tag[labels[idx]])
            sentence_predictions.append(idx2tag[predictions[idx]])
            if masks[idx] == 1:
                sentence_true_labels_mask.append(idx2tag[labels[idx]])
                sentence_pred_labels_mask.append(idx2tag[predictions[idx]])
            sentence_length += 1

            if sentence_length == batch_element_length:
                # not including the 0 and the last element of list, because of CLS and SEP tokens
                clear_labels.append(sentence_labels[1: len(sentence_labels) - 1])
                clear_predictions.append(sentence_predictions[1: len(sentence_predictions) - 1])
                masked_true_labels.append(sentence_true_labels_mask[1: len(sentence_true_labels_mask) - 1])
                masked_pred_labels.append(sentence_pred_labels_mask[1: len(sentence_pred_labels_mask) - 1])
                sentence_labels = []
                sentence_predictions = []
                sentence_true_labels_mask = []
                sentence_pred_labels_mask = []
                sentence_length = 0
        else:
            if sentence_labels:
                clear_labels.append(sentence_labels[1: len(sentence_labels) - 1])
                clear_predictions.append(sentence_predictions[1: len(sentence_predictions) - 1])
                masked_true_labels.append(sentence_true_labels_mask[1: len(sentence_true_labels_mask) - 1])
                masked_pred_labels.append(sentence_pred_labels_mask[1: len(sentence_pred_labels_mask) - 1])
                sentence_labels = []
                sentence_predictions = []
                sentence_true_labels_mask = []
                sentence_pred_labels_mask = []
            else:
                pass

    masked_true_labels = [element for element in masked_true_labels if element != []]
    masked_pred_labels = [element for element in masked_pred_labels if element != []]
    repeated_entities_labels = {'true': masked_true_labels, 'pred': masked_pred_labels}

    return clear_labels, clear_predictions, repeated_entities_labels

def train_epoch(model, criterion, optimizer, data, tag2idx, idx2tag, device, scheduler=None, name=None):
    epoch_loss = 0
    epoch_metrics = FMeasureStorage()
    epoch_repeated_entities_accuracy = AccuracyStorage()

    model.train()
    with tqdm(total=len(data)) as progress_bar:
        for batch in data:
            tokens = batch[0].to(device)
            tags = batch[1].to(device)
            masks = batch[2]

            batch_element_length = len(tags[0])

            predictions = model(tokens)
            predictions = predictions.view(-1, predictions.shape[-1])

            tags_mask = tags != tag2idx['<PAD>']
            tags_mask = tags_mask.view(-1)
            labels = torch.where(tags_mask, tags.view(-1), torch.tensor(criterion.ignore_index).type_as(tags))

            masks = masks.view(-1)

            loss = criterion(predictions, labels)

            predictions = predictions.argmax(dim=1)

            predictions = predictions.cpu().numpy()
            labels = labels.cpu().numpy()
            masks = masks.cpu().numpy()

            # clear <PAD>, CLS and SEP tags from both labels and predictions
            clear_labels, clear_predictions, clear_repeated_entities_labels = clear_tags(labels, predictions, masks, idx2tag, batch_element_length)

            iteration_result = performance_measure(clear_labels, clear_predictions)

            epoch_metrics + iteration_result
            epoch_repeated_entities_accuracy + clear_repeated_entities_labels
            epoch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
            if scheduler:
                scheduler.step()
            torch.cuda.empty_cache()

            progress_bar.update()
            progress_bar.set_description('{:>5s} Loss = {:.5f}, F1-score = {:.2%}, Repeated Entities Accuracy = {:.2%}'.format(name, loss.item(), 0, 0))

        epoch_f1_score, epoch_precision, epoch_recall = epoch_metrics.report()
        epoch_accuracy = epoch_repeated_entities_accuracy.report()
        progress_bar.set_description('{:>5s} Loss = {:.5f}, F1-score = {:.2%}, Repeated Entities Accuracy = {:.2%}'.format(name, epoch_loss / len(data), epoch_f1_score, epoch_accuracy))


def eval_epoch(model, criterion, data, tag2idx, idx2tag, device, name=None):
    epoch_loss = 0
    epoch_metrics = FMeasureStorage()
    epoch_repeated_entities_accuracy = AccuracyStorage()

    model.eval()

    with torch.no_grad():
        with tqdm(total=len(data)) as progress_bar:
            for batch in data:
                tokens = batch[0].to(device)
                tags = batch[1].to(device)
                masks = batch[2]

                batch_element_length = len(tags[0])

                predictions = model(tokens)
                predictions = predictions.view(-1, predictions.shape[-1])
                tags_mask = tags != tag2idx['<PAD>']
                tags_mask = tags_mask.view(-1)
                labels = torch.where(tags_mask, tags.view(-1), torch.tensor(criterion.ignore_index).type_as(tags))

                masks = masks.view(-1)

                loss = criterion(predictions, labels)

                predictions = predictions.argmax(dim=1)

                predictions = predictions.cpu().numpy()
                labels = labels.cpu().numpy()
                masks = masks.cpu().numpy()

                # clear <PAD>, CLS and SEP tags from both labels and predictions
                clear_labels, clear_predictions, clear_repeated_entities_labels = clear_tags(labels, predictions, masks, idx2tag, batch_element_length)

                iteration_result = performance_measure(clear_labels, clear_predictions)

                epoch_metrics + iteration_result
                epoch_repeated_entities_accuracy + clear_repeated_entities_labels
                epoch_loss += loss.item()

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, F1-score = {:.2%}, Repeated Entities Accuracy = {:.2%}'.format(name, loss.item(), 0, 0))

            epoch_f1_score, epoch_precision, epoch_recall = epoch_metrics.report()
            epoch_accuracy = epoch_repeated_entities_accuracy.report()
            progress_bar.set_description('{:>5s} Loss = {:.5f}, F1-score = {:.2%}, Repeated Entities Accuracy = {:.2%}'.format(name, epoch_loss / len(data), epoch_f1_score, epoch_accuracy))
        


def train_model(model, criterion, optimizer, train_data, eval_data, tag2idx, idx2tag, device, scheduler, epochs=1):
    for epoch in range(epochs):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs)
        train_epoch(model, criterion, optimizer, train_data, tag2idx, idx2tag, device, scheduler, name_prefix + 'Train:')
        eval_epoch(model, criterion, eval_data, tag2idx, idx2tag, device, name_prefix + 'Eval :')

In [11]:
import torch.optim as optim
import torch
import torch.nn as nn

classes = len(train_dataset.ner_tags)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BertNER(classes).to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)

EPOCHS = 4

train_model(model, criterion, optimizer, train_dataloader, eval_dataloader, train_dataset.tag2idx, train_dataset.idx2tag, device, None, EPOCHS)
test_model(model, criterion, test_dataloader, train_dataset.tag2idx, train_dataset.idx2tag, device)

[1 / 4] Train: Loss = 0.14119, F1-score = 81.78%, Repeated Entities Accuracy = 75.96%: 100%|██████████| 6348/6348 [22:22<00:00,  4.73it/s]
[1 / 4] Eval : Loss = 0.12821, F1-score = 86.42%, Repeated Entities Accuracy = 80.23%: 100%|██████████| 865/865 [00:47<00:00, 18.12it/s]
[2 / 4] Train: Loss = 0.09491, F1-score = 87.85%, Repeated Entities Accuracy = 84.88%: 100%|██████████| 6348/6348 [22:34<00:00,  4.69it/s]
[2 / 4] Eval : Loss = 0.12860, F1-score = 86.79%, Repeated Entities Accuracy = 81.03%: 100%|██████████| 865/865 [00:47<00:00, 18.28it/s]
[3 / 4] Train: Loss = 0.07812, F1-score = 89.95%, Repeated Entities Accuracy = 88.25%: 100%|██████████| 6348/6348 [22:34<00:00,  4.69it/s]
[3 / 4] Eval : Loss = 0.12365, F1-score = 87.09%, Repeated Entities Accuracy = 81.68%: 100%|██████████| 865/865 [00:47<00:00, 18.29it/s]
[4 / 4] Train: Loss = 0.07267, F1-score = 90.87%, Repeated Entities Accuracy = 89.51%: 100%|██████████| 6348/6348 [22:37<00:00,  4.68it/s]
[4 / 4] Eval : Loss = 0.13371, F1